<a href="https://colab.research.google.com/github/rohit-447/Py-Torch-Learning/blob/main/Pytorch_CNN_Food101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working on Food101 Dataset as Custom Dataset


In [ ]:
#@title Import of modules
import torch
from torch import nn
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import Food101
import matplotlib.pyplot as plt
from pathlib import Path
torch.__version__, torchvision.__version__

In [ ]:
#@title Download of DataSet
FOOD101=Food101(root='data', download=True)

In [ ]:
#@title Walk through Directory
import os
def walk_through_dir(dir_path):
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} files in {dirpath}")

#going through folder
PATH=Path('data')
walk_through_dir(PATH)

In [ ]:
#@title Split of Dataset into Train and Test and save them.
from sklearn.model_selection import train_test_split
import os
import shutil
import tqdm.auto as tqdm
DATA_ORIGINAL_PATH='data/food-101'
IMAGES='images'
DATA_SPLIT_PATH='Dataset_new'
if not os.path.exists(DATA_SPLIT_PATH):
  os.makedirs(DATA_SPLIT_PATH, exist_ok=True)

source_dir=os.path.join(DATA_ORIGINAL_PATH,IMAGES)
train_dir=os.path.join(DATA_SPLIT_PATH, 'train')
test_dir=os.path.join(DATA_SPLIT_PATH, 'test')
val_dir=os.path.join(DATA_SPLIT_PATH, 'val')

#clear the dir if it exists
for dir_path in [train_dir, test_dir, val_dir]:
  if os.path.exists(dir_path):
    shutil.rmtree(dir_path)
  os.makedirs(dir_path)

#Iterrate over the classes of images
for  class_name in tqdm.tqdm(os.listdir(source_dir), desc="Processing Classes", unit="class"):
  class_path= os.path.join(source_dir, class_name)
  if not os.path.isdir(class_path):
    continue

  #list all images in class folder
  images=[os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith('.jpg')]
  dataset_train, dataset_test=train_test_split(images, test_size=0.2, random_state=42)

  #copy the train test files and save them
  def copy_files(file_list, save_path):
    save_root=os.path.join(save_path, class_name)
    os.makedirs(save_root, exist_ok=True)
    for f in file_list:
      shutil.copy(f, os.path.join(save_root, os.path.basename(f)))

  copy_files(dataset_train, train_dir)
  copy_files(dataset_test, test_dir)
print(f'Dataset Suceesfully split into Train and Test')

In [ ]:
#checking dataset Split
walk_through_dir(DATA_SPLIT_PATH)

In [ ]:
#@title Visulisation of Random Images
import random
import PIL
import glob
DATASET=Path('Dataset_new')
random_image=list(DATASET.glob('*/*/*.jpg'))
random_image_path=random.choice(random_image)
random_image_class=random_image_path.parent.stem
img=PIL.Image.open(random_image_path)
print(f'Image Path: {random_image_path}')
print(f'Class Height: {img.height} and Width: {img.width}')
img

In [ ]:
#@title Device Agonistic Code
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device